In [2]:
import requests
import time
import re
import pickle
import openai
from bs4 import BeautifulSoup

# Robots.txt

The following code reads the robots.txt file and creates a dictionary. SUNY Brockport website only contains Disallowed, so we are concerned with that alone. Will use regex to parse links and make sure we don't crawl these wepages

In [3]:
import os
result = os.popen("curl https://www2.brockport.edu/robots.txt").read()
result_data_set = {"Disallowed":[], "Allowed":[]}

for line in result.split("\n"):
    if line.startswith('Allow'):    # this is for allowed url
        result_data_set["Allowed"].append(line.split(': ')[1].split(' ')[0])    # to neglect the comments or other junk info
    elif line.startswith('Disallow'):    # this is for disallowed url
        result_data_set["Disallowed"].append(line.split(': ')[1].split(' ')[0])    # to neglect the comments or other junk info

disallow_list = result_data_set['Disallowed']

# Adding my own filters. These files tend to not work throughout.
disallow_list.extend(['/live/files/', '.html/'])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1048  100  1048    0     0   6748      0 --:--:-- --:--:-- --:--:--  6761


# Seach_page

Function to pull webpages from all links on a webpage given by a link

Steps:
- User provides link
- Get the corresponding webpage for said link
- Come up with a list of all links on that webpage
- Get the corresponding webpage for each of those links

In [4]:
def search_page(link: str) -> dict:
    data = {}

    # Get the base page and parse with beautiful soup
    data[link] = requests.get(link)
    soup = BeautifulSoup(data[link].text, 'html.parser')

    # Find all the links on the page
    links = soup.find_all('a')

    # Filter the links to get only those that point to other pages on the same site
    # This is done by checking if the link starts with '/' (indicating it's a relative link)
    # and doesn't contain '#' (which would indicate it's a link to a specific part of the same page)
    links = [link.get('href') for link in links if link.get('href').startswith('/') and '#' not in link.get('href')]

    # APPLY FILTER... Gets rid of robots.txt files, and some others.
    links = [link for link in links if not any(bad_link in link for bad_link in disallow_list)]

    # Remove duplicates
    links = list(set(links))

    # loop over the links and scrape each page
    for link in links:
        try:
            # Construct the full URL by appending the link to the base URL
            full_url = "https://www2.brockport.edu" + link
            data[full_url] = requests.get(full_url)

            # To not overload the website, sleep 1 sec
            time.sleep(1)
        except Exception as e:
            print(f"An error occurred when processing {full_url}")
            continue

    return data

In [5]:
data = search_page('https://www2.brockport.edu/')

An error occurred when processing https://www2.brockport.edu/directory/
An error occurred when processing https://www2.brockport.edu//www.instagram.com/brockport
An error occurred when processing https://www2.brockport.edu//www.twitter.com/brockport
